In [1]:
!pip install transformers

In [10]:
import math
import numpy as np
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model

In [11]:
import pandas as pd

We'll read the tweet file


In [12]:
clim = pd.read_csv('Climate_twitter.csv')
bt = pd.read_csv('twitter_sentiment_data.csv')
json = pd.read_json('train.jsonl', lines=True)

In [13]:
# adding label based on polarity score
conditions = [
    (clim['polarity'] < 0),
    (clim['polarity'] > 0),
    (clim['polarity'] == 0)]

values = [0,1, 'neutral']

clim['label'] = np.select(conditions, values)

clim.head()

,id,date,retweets,source,author,likes,text,twitter_name,location,verified,followers,friends,polarity,subjectivity,label
0,2184934963,2020-12-22 23:22:20,71,Twitter Web App,GO GREEN,91,The death of summer Arctic ice our Earth coole...,ECOWARRIORSS,NaN,False,23415,20439,-0.054365,0.426984,0
1,508658626,2020-12-10 14:30:00,14,Twitter for Advertisers,Elsevier Energy,98,Elsevier and the EditorsinChief are pleased to...,ElsevierEnergy,"Oxford, England",False,6615,508,0.387500,0.633333,1
2,2607105006,2020-12-22 21:28:52,0,Twitter Web App,Arwyn Thomas,1,From better climate change education to improv...,siwarr5,Carmarthen,False,22,133,0.261905,0.345238,1
3,19609660,2020-12-22 21:24:10,0,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",0,climate change Links to FIXING CLIMATE CHANGE ...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000,neutral
4,19609660,2020-12-21 22:52:09,1,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",1,climate change The 11TH HOUR FOR THE EARTH cli...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000,neutral


In [14]:
# dropping neutral entries
clim = clim.drop(clim[clim['label'] == 'neutral'].index)

In [15]:
# dropping all columns except text and label
clim = clim.drop(columns = ['id','date','source','author','twitter_name','location','verified','retweets','likes','followers','friends','polarity','subjectivity'])

In [16]:
clim.info()
clim['text'] = clim['text'].astype('string')
clim['label'] = clim['label'].astype('int64')
clim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 395
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    270 non-null    object
 1   label   270 non-null    object
dtypes: object(2)
memory usage: 6.3+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 395
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    270 non-null    string
 1   label   270 non-null    int64 
dtypes: int64(1), string(1)
memory usage: 6.3 KB


In [17]:
bt.head()
bt = bt.drop(columns=['tweetid'])

In [18]:
bt.rename(columns={'sentiment': "label", 'message':'text'}, inplace=True)

In [19]:
bt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43943 entries, 0 to 43942
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   43943 non-null  int64 
 1   text    43943 non-null  object
dtypes: int64(1), object(1)
memory usage: 686.7+ KB


In [20]:
bt['label'].value_counts() # imbalance

 1    22962
 2     9276
 0     7715
-1     3990
Name: label, dtype: int64

In [21]:
bt = bt.drop(bt[bt['label'] ==  2].index)

In [22]:
bt = bt.drop(bt[bt['label'] == 0].index)

In [23]:
bt.dropna(subset=['label'], inplace=True)

In [24]:
conditions = [
    (bt['label'] == -1),
    (bt['label'] == 1)]

values = [0,1]

bt['sent'] = np.select(conditions, values)

In [25]:
bt['label'] = bt['sent']
bt=bt.drop(columns=['sent'])

In [26]:
bt['text'] = bt['text'].astype("string")
bt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26952 entries, 0 to 43942
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   26952 non-null  int32 
 1   text    26952 non-null  string
dtypes: int32(1), string(1)
memory usage: 526.4 KB


In [27]:
bt['label'].value_counts()

1    22962
0     3990
Name: label, dtype: int64

In [28]:
json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2117 entries, 0 to 2116
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2117 non-null   object
 1   label   2117 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.2+ KB


In [29]:
json['label'].value_counts()

0    1585
1     532
Name: label, dtype: int64

In [30]:
df_list = [clim, bt, json]
df = pd.concat(df_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29339 entries, 0 to 2116
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29339 non-null  object
 1   label   29339 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 687.6+ KB


In [31]:
df['text'] = df['text'].astype('string')

In [32]:
df['label'].value_counts()

1    23684
0     5655
Name: label, dtype: int64

In [33]:
import nltk

# Download the lexicon
nltk.download("vader_lexicon")

# Import the lexicon
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create an instance of SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [34]:
nltk.download('punkt')  # Download the punkt tokenizer if not already downloaded

df['tokens'] = df['text'].apply(nltk.word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
keywords = ["sustainability", "environmental", "conservation", "recycling", "sustainable",
                         "responsibility", "green", "eco-friendly", "renewable", "carbon", "climate", "ecosystem",
                         "planet", "biodiversity", "energy", "water", "pollution", "reduction", "renewability",
                         "ecological", "renewable", "greenhouse", "clean", "solar", "wind", "earth", "sustainable",
                         "planet", "ecology", "ocean", "forest", "organic", "earth-friendly", "bio", "ethics",
                         "conservationist", "sustain", "renew", "ethical", "greenery", "saver", "sustainable",
                         "conservator", "recycler", "biodegradable", "natural", "greenery", "environment",
                         "saver", "earth-saving", "sustainability", "green-living", "clean", "responsible",
                         "preservation", "regeneration", "ecosystem", "safeguarding"]
filtered_df = df[df['tokens'].apply(lambda tokens: any(keyword in tokens for keyword in keywords))]

In [36]:
filtered_df['label'].value_counts()

1    19287
0     2412
Name: label, dtype: int64

In [37]:
from sklearn.utils import resample
df1 = filtered_df.loc[filtered_df['label'] == 0]
df2 = filtered_df.loc[filtered_df['label'] == 1]
df1_sampled=resample(df1, replace=True,
                     n_samples= 19287,
                     random_state=42)
# reproducible results
filtered_df= pd.concat([df1_sampled, df2])
filtered_df.info()
filtered_df['label'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38574 entries, 20004 to 2115
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    38574 non-null  string
 1   label   38574 non-null  int64 
 2   tokens  38574 non-null  object
dtypes: int64(1), object(1), string(1)
memory usage: 1.2+ MB


0    19287
1    19287
Name: label, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
df.columns

Index(['text', 'label', 'tokens'], dtype='object')

Split it into train and test.

In [40]:
X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [41]:
X_train

33       We've ordered 25, and we expect to continue to...
42753    RT @AC_screenwriter: Climate change is here an...
39989    RT As The GOP Was Calling Obama A Weak Leader,...
20181    RT @mslopatto: If you want actual accurate rep...
4675     RT @SenSanders: We have a president-elect who ...
                               ...                        
17851    Trumps actions on climate change are an assaul...
17875    RT @GreenStar_UK: There has always been reason...
40680    Changing climate change starts with you. Sign ...
41832    RT @TheFederalist1: New Great Lakes Study Blow...
1618     And in government services, there is both supp...
Name: text, Length: 22004, dtype: string

We will assign the max length of the sequence as the average length of all the tweets + 2 (arbitrary)

In [42]:
MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2
MAX_LENGTH

22

Let's initialize the tokenizer and add a PAD token ane and EOS (end of text) token

In [43]:
PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

# this will download and initialize the pre trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\sayan\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sayan\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

We add the EOS token at the end of each tweet

In [44]:
X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

We pass the tweets through the tokenizer, padding them to the max_length. This is a list so we need to stack them to create a tensor input (using tf.convert_to_tensor)

In [45]:
X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

C:\Users\sayan\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [46]:
X_train_in[0]

<tf.Tensor: shape=(22,), dtype=int32, numpy=
array([ 1135,  1053,  6149,  1679,    11,   290,   356,  1607,   284,
        2555,   284,  1325,  1111,   832, 12673,    11,   355,   880,
         355,  4896,   287, 17173])>

We will also get the mask from the tokenizer (1 is token, 0 is pad token) the same we did for the inputs.

In [47]:
X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)

We are now ready to pass the inputs through the model.
Next we need to initilize GPT2 for TF and set it in training mode.

In [48]:
model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [49]:
model.resize_token_embeddings(len(tokenizer))

Set the GPT2 pre-trained layers as non trainable

In [50]:
for layer in model.layers:
    layer.trainable = False

In [51]:
model.summary()

Model: "tfgpt2_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 124440576 
 r)                                                              
                                                                 
Total params: 124,440,576
Trainable params: 0
Non-trainable params: 124,440,576
_________________________________________________________________


Next we build on top of GPT2.
The model takes in tokens and mask tensors. The outputs are the last hidden states of the last layer in the transformer. These are reduced using the mean over the sequence length, passed through 2 dense layers with dopout in between. The output layer has three node (softmax activation function for probabilities) for the three classes we want to predict (Negative, Neutral and Positive).

In [52]:
input = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input, attention_mask=mask)
#x = x.last_hidden_state[:, -1]
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

In [53]:
clf = tf.keras.Model([input, mask], output)

In [54]:
clf.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 tfgpt2_model (TFGPT2Model)     TFBaseModelOutputWi  124440576   ['input_1[0][0]',                
                                thPastAndCrossAtten               'input_2[0][0]']                
                                tions(last_hidden_s                                               
                                tate=(None, None, 7                                           

Next we compile the model choosing the learning rate, loss function and the metric to monitor. Also a callback function.

In [55]:
base_learning_rate = 0.0005
optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
#loss=tf.keras.losses.BinaryCrossentropy()
loss=tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [56]:
callbacks = tf.keras.callbacks.EarlyStopping(
        monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

Last thing we need to do is preparing the target tensor.

In [57]:
#def map_sentiment(value):
 # if value == 'Negative':
  #  return 0
  #if value == 'Neutral':
  #  return 1
  #if value == 'Positive':
  #  return 2

In [60]:
def map_sentiment(value):
  if value == 0:
    return 0
  else:
    return 1

In [61]:
y_train_ = y_train.map(map_sentiment)
y_test_ = y_test.map(map_sentiment)

In [62]:
y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)

In [63]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


We then train the model passing number of epochs, batch_size and validation split.

In [ ]:
history = clf.fit([X_train_in, X_train_mask], y_train_in, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)

Epoch 1/30


C:\Users\sayan\Anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


160/551 [=======>......................] - ETA: 15:41 - loss: 1.9533 - accuracy: 0.6682

We can now evaluate the model on the test set.

In [ ]:
clf.evaluate([X_test_in, X_test_mask], y_test_in)

In [ ]:
clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

In [ ]:
y_pred_out = tf.math.argmax(y_pred, axis=-1)
y_pred_out

We can now evaluate the model on the test set using the classification report and confusion matrix.

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test_in, y_pred_out))

We notice right away that the model, overall, has a good precision and recall on all classes. The model has more difficulties in predicting the negative class (0), expecially recall.

Of all the predicted negative, 60% were actually negative - the model has a decent precision. Of all the ones that were actually negative, only 56% were predicted negative - the model struggles recalling (recall metric) the negative class.

Below is the confusion matrix that helps understand it better.

In [ ]:
confusion_df = pd.DataFrame(confusion_matrix(y_test_in, y_pred_out))
confusion_df.index = ['Actual 0', 'Actual 1']
confusion_df.columns = ['Predicted 0', 'Predicted 1']
confusion_df

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(confusion_df, annot=True, fmt='d', linewidths=0.5) 
plt.yticks(rotation=0)
plt.xticks(rotation=45)
plt.show()

Overall the model doesn't seem to outperform logistic regression and random forest. 
It's worth noticing that the model is overfitting so it would be nice to re-train it, playing with learning rate, weight decay and dropout rate.  

In [ ]:
#clf.save_weights('/content/gdrive/My Drive/Data Science/saved_model/GPT2_transfer_learning')